# Encounters between ants
Function to calculate the encounters between specific ants (e.g., focal and caregiver ants) from a ```.mymridon``` experiment file. <br><br>
There is probably an easier way to do this by querying individual frames directly.

In [ ]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np
import pandas as pd  # Used to create a dataframe, similar to the structure used in R
import py_fort_myrmidon as fm

### Function to output trajectories of all ants

In [ ]:
def trajectory_output_all(start_time, end_time, exp):
    """
    Function to extract daily trajectories as a parquet file, grouped by AntID. While it is setup to extract daily trajectories, it can work for any arbitrary time duration
    :param start_time: The start datetime object. this will be converted to a fort-myrmidon Time object
    :param end_time: The end datetime object. this will be converted to a fort-myrmidon Time object
    :param exp: The name of the experiment i.e., the myrmidon file
    :param matcher_query: The fm matcher corresponding to the focal IDs
    :return: Outputs a pandas dataframe containing AntID, Space, Time, X_coordinates and Y_coordinates of each ID averaged over 1 second from the X and Y coordinates. Averagingg is done to have a dataset which can be merged across IDs using at the resolution of 1s.
    """
    start = datetime.now()
    t_begin = fm.Time(start_time)
    t_stop = fm.Time(end_time)
    trajectory = fm.Query.ComputeAntTrajectories(
        experiment=exp,
        start=t_begin,
        end=t_stop,
        # matcher=matcher_query,
        maximumGap=fm.Duration.Parse("1000h"),
        reportProgress=False,
    )
    # Make a list of lists with trajectory values needed. Position is an array of 5 columns, so specific columns are called
    traj_list = [
        [
            trajectory.Ant,
            trajectory.Space,
            trajectory.Start.ToDateTime(),
            trajectory.Positions[:, 0],
            trajectory.Positions[:, 1],
            trajectory.Positions[:, 2],
        ]
        for trajectory in trajectory
    ]
    # Make the list into a dataframe
    traj_df = pd.DataFrame(
        traj_list,
        columns=["AntID", "Space", "StartTime", "Pos_time", "X_coor", "Y_coor"],
    )
    # Explode columns which are in the form of lists to expand the dataframe
    traj_df = traj_df.explode(column=["Pos_time", "X_coor", "Y_coor"])
    # Coerce coordinates to integer
    traj_df["X_coor"] = pd.to_numeric(traj_df["X_coor"], errors="coerce")
    traj_df["Y_coor"] = pd.to_numeric(traj_df["Y_coor"], errors="coerce")
    # Convert Pos_time to timedelta and obtain actual datetime for each trajectory entry
    traj_df["Pos_time"] = pd.to_numeric(traj_df["Pos_time"], errors="coerce")
    traj_df["Pos_time"] = pd.to_timedelta(
        traj_df["Pos_time"], unit="S", errors="coerce"
    )
    traj_df["Time"] = traj_df["StartTime"] + traj_df["Pos_time"]
    # Drop unwanted ccolumns
    traj_df = traj_df.drop(["StartTime", "Pos_time"], axis=1)
    # Reorder columns
    traj_df = traj_df[["AntID", "Space", "Time", "X_coor", "Y_coor"]]
    if traj_df.empty:  # If no trajectories are output
        # empty_row = pd.DataFrame([{'AntID': 'Unknown', 'Space':np.nan, 'Time':np.nan, 'X_coor':np.nan, 'Y_coor':np.nan}]) # Create empty row with unknown as antID
        # traj_df = pd.concat([empty_row]) # Add empty row to dataframe
        print("No trajectories found. Created empty dataframe")
        return traj_df  # Return empty dataframe
    # Obtain average X and Y coordinates per second
    traj_df = (
        traj_df.groupby([pd.Grouper(key="Time", freq="1s"), "AntID", "Space"])
        .agg(X_mean=("X_coor", "mean"), Y_mean=("Y_coor", "mean"))
        .reset_index()
    )
    end = datetime.now()
    # print("Trajectories output in", end-start)
    return traj_df

### Function to count encounters

In [ ]:
def count_encounters(displacement_dataset, encounter_threshold, away_threshold):
    """
    Function to calculate number of encounters between caregivers and focal ants. Function should be run only on datasets which contain displacement data from one pair of caregiver and focalID, since it is based on the index values. Either a subset dataset obtained from focal_caregivers_disp_exp can be used, or the function can be applied after grouping by specific variables in a larger dataframe
    :param displacement_dataset: The displacement dataset; either a subset obtained from focal_caregivers_disp_exp or a pandas grouped by dataset
    :param encounter_threshold: Threshold displacement to use as encounter
    :param away_threshold: Threshold displacement to use as the start/end of an encounter
    :return:
    """
    # Recreate displacement to include values across different spaces (since we need it to not be np.nan to distinguish it from actual unknown values)

    # Create a new column based on converting the thresholds to dummy numbers. Values given are 1, if displacement < encounter threshold, 0.5, if encounter threshold < disp < away threshold and 0 if disp > away threshold. np.nans are retained
    displacement_dataset["enc_comb"] = pd.cut(
        displacement_dataset.disp,
        [0, encounter_threshold, away_threshold, np.inf],
        labels=[1, 0.5, 0],
    )
    # Convert datatype to float from category (due to pd.cut) for downstream functions
    displacement_dataset = displacement_dataset.astype({"enc_comb": float})

    if (
        displacement_dataset.enc_comb.isnull().all()
    ):  # Check if all values in enc_comb are NA
        print("No displacement data available")
        encounters = "NA"  # If enc_comb has only na values, make encounters = NA
        return encounters
    enc_ind = displacement_dataset[displacement_dataset["enc_comb"] == 1].index.values
    if enc_ind.size == 0:  # Check if the number of instances of 1 in enc_comb is 0
        encounters = 0  # If enc_comb has no 1 value, make encounters = 0 and exit
        return encounters

    # print(f"{'exp_day is'}{displacement_dataset.exp_day.unique()}{'and caregiverID is'}{displacement_dataset.caregiverID.unique()}")
    if enc_ind[0] != np.take(
        displacement_dataset.index.values, 0
    ):  # Check if first value is not starting index
        enc_ind = np.insert(
            enc_ind, 0, np.take(displacement_dataset.index.values, 0)
        )  # Insert starting value of index range
    enc_ind_pair = list(zip(enc_ind, enc_ind[1:]))  # Get pairs of consecutive indices
    enc_comb_list = [
        displacement_dataset.loc[x + 1 : y - 1, "enc_comb"] for (x, y) in enc_ind_pair
    ]  # Get values between the consecutive indices with value of 1
    list_filter = [
        np.logical_not(x.size < 2) for x in enc_comb_list
    ]  # Get Boolean list of where the individual list size is greater than 2
    enc_comb_sub = [
        enc_comb_list[i] for i in range(len(enc_comb_list)) if list_filter[i]
    ]  # Use the boolean list to filter the original list
    enc_comb_list_nona = [
        x[np.logical_not(np.isnan(x))] for x in enc_comb_sub
    ]  # Remove nas from the list
    enc_comb_zero = [
        np.any(x) for x in enc_comb_list_nona
    ]  # Get True if any element in the list is 0
    encounters = sum(enc_comb_zero)  # Get summation of number of elements with 0
    return encounters

### Function to count encounters of specific ants

This function combines obtaining all the trajectories from the `trajectory_output_all` function, then rearranges this to obtain the displacement between the focal ant and all other ants at each time point (in this every second, since the trajectories are summarised to the nearest second by averaging the X and Y coordinate for each second). This dataset is then grouped by focalID and antID and then the number of encounters between these 2 IDs are calculated

In [ ]:
def focal_encounters(
    start_time, end_time, exp, focal_ID, exp_day, encounter_threshold, away_threshold
):
    """
    Function to obtain trajectories for focal and caregiver antIDs, merge by time and calculate displacement of each caregiver ID from the focal ID at every second
    :param start_time: Starting time to obtain trajectories from. Passed on to function trajectory_output
    :param end_time: Ending time to obtain trajectories from. Passed on to function trajectory_output
    :param exp: Location of myrmidon file
    :param focal_ID: Injured AntID
    :param exp_day: Day of the experiment. This is added to the dataframe for identification
    :param encounter_threshold: Threshold displacement to use as encounter
    :param away_threshold: Threshold displacement to use as the start/end of an encounter
    :return: Returns a datafarme containing the Time (in bins of 1s based on function trajectory_output), the focal and caregiver ID, the space in which the focal and caregiver ants are present, and the displacement between them (calculated as np.nan if they are in different spaces. In a CSV output this will be converted to a blank entry).
    """
    start = datetime.now()
    # # Focal Ant matcher
    # focal_matcher = fm.Matcher.AntID(focal_ID)
    # # Caregiver individual matchers
    # # others = [fm.Matcher.AntID(x) for x in other_IDs]
    # # Create single matcher object by unpacking the list within an Or Matcher
    # #others_matcher = fm.Matcher.Or(*others)
    # # Focal Ant trajectory
    # focal_traj = trajectory_output(start_time, end_time, exp, focal_matcher)
    # All ant trajectories
    other_traj = trajectory_output_all(start_time, end_time, exp)
    # Focal ant trajectory
    focal_traj = other_traj[other_traj["AntID"] == focal_ID]
    # Merge focal and caregiver trajectories on Time column
    if focal_traj.empty:  # If focal trajectory is an empty dataframe
        full_traj = other_traj.rename(columns={"Space": "Space_ant"})
        full_traj["focalID"] = focal_ID
        full_traj["Space_focal"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
        ]
        full_traj["exp_day"] = exp_day
        full_traj = full_traj[full_traj["focalID"] != full_traj["AntID"]].reset_index()
        enc = (
            full_traj.groupby(["exp_day", "focalID", "AntID"])
            .apply(lambda x: "NA")
            .reset_index()
            .rename(columns={0: "encounters"})
        )  # Apply count_encounters function over grouped dataframe, reset index and rename columns
        print(
            f"{'Focal ID trajectory is empty for list item '}{exp_day}{' .Returning dataframe with no displacement and encounters calculated'}"
        )
        return enc
    if other_traj.empty:  # If caregivers trajectory is an empty dataframe
        full_traj = focal_traj.rename(
            columns={"AntID": "focalID", "Space": "Space_focal"}
        )
        full_traj["AntID"] = full_traj["Space_ant"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
        ]
        full_traj["exp_day"] = exp_day
        enc = (
            full_traj.groupby(["exp_day", "focalID", "AntID"])
            .apply(lambda x: "NA")
            .reset_index()
            .rename(columns={0: "encounters"})
        )  # Apply count_encounters function over grouped dataframe, reset index and rename columns
        print(
            f"{'Caregiver ID trajectories are empty for list item '}{exp_day}{' .Returning dataframe with no displacement and encounters calculated'}"
        )
        return enc
    full_traj = pd.merge(
        other_traj, focal_traj, how="outer", on="Time", suffixes=("_ant", "_focal")
    )
    # Obtain X coordinate and Y coordinate difference between Focal and Caregivers, for each row
    full_traj["X_diff"] = full_traj["X_mean_focal"] - full_traj["X_mean_ant"]
    full_traj["Y_diff"] = full_traj["Y_mean_focal"] - full_traj["Y_mean_ant"]
    # Obtain displacement
    full_traj["disp"] = np.linalg.norm(
        full_traj[["X_diff", "Y_diff"]].to_numpy(), axis=1
    )
    full_traj = full_traj.rename(
        columns={"AntID_focal": "focalID", "AntID_ant": "AntID"}
    )
    full_traj = full_traj[
        ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
    ]
    full_traj["exp_day"] = exp_day
    # Remove instances where the focal ant's displacement is calculated wrt itself.
    full_traj = full_traj[full_traj["focalID"] != full_traj["AntID"]].reset_index()
    # Replace with arbitrary high value of displacemeent if focal ant and caregiver are in different spaces. Use notnull to filter out instances where focal or caregiver space is not known. The higgh value will ensure that this case is always considered as > away_threshold in count_encounters function
    full_traj.loc[
        (
            (full_traj.Space_focal.notnull())
            & (full_traj.Space_ant.notnull())
            & (full_traj.Space_focal != full_traj.Space_ant)
        ),
        "disp",
    ] = 100000
    enc = (
        full_traj.groupby(["exp_day", "focalID", "AntID"])
        .apply(lambda x: count_encounters(x, encounter_threshold, away_threshold))
        .reset_index()
        .rename(columns={0: "encounters"})
    )  # Apply count_encounters function over grouped dataframe, reset index and rename columns
    end = datetime.now()
    print(f"{'Displacement for list item '}{exp_day}{' calculated in '}{end - start}")
    return enc

### Function to run encounter calculations over multiple phases of one experiment

In [ ]:
def calculate_encounters_cluster(exp_list, control_list, pre_list, post_list, colonyID):
    """
    Helper function to run focal_caregivers_disp_exp and count_encounters over multiple lists associated with different phases for one colony
    :param exp_list: List of start_time, end_time, exp, focalID, careggiverIDs and exp_day for Experimental phase
    :param control_list: List for control phase
    :param pre_list: List for pre-experimental phase
    :param post_list: List for post-experimental phase
    :param colonyID: colonyID corresponding to the myrmidon file
    :return: Dataframe combining the encounters for each of the 4 phases for one colony
    """
    print("Experimental phase")
    exp_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in exp_list
    ]
    exp_enc = pd.concat(exp_enc)
    exp_enc["phase"] = "Exp"
    print("Control phase")
    control_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in control_list
    ]
    control_enc = pd.concat(control_enc)
    control_enc["phase"] = "Control"
    print("Pre-Experimental phase")
    pre_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in pre_list
    ]
    pre_enc = pd.concat(pre_enc)
    pre_enc["phase"] = "Pre"
    print("Post-Experimental phase")
    post_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in post_list
    ]
    post_enc = pd.concat(post_enc)
    post_enc["phase"] = "Post"
    # Combine encounter dataframes
    enc_list = [exp_enc, control_enc, pre_enc, post_enc]
    enc = pd.concat(enc_list)
    # Add colonyID
    enc["colony"] = colonyID
    # Sort by values
    enc = enc.sort_values(
        by=["colony", "phase", "exp_day", "focalID", "AntID"]
    ).reset_index(drop=True)
    enc = enc[["colony", "phase", "exp_day", "focalID", "AntID", "encounters"]]
    return enc

## Colony Cfel 42

In [ ]:
f_myrmidon = "/media/egeorge/Elements/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)

# Thresholds
encounter_threshold = 300  # threshold for counting as an encounter
away_threshold = 1000  # Threshold for counting as the end of an encounter

In [ ]:
# To run the function over all the ants, we will need to get the list of antIDs
# ants = list(exp.Ants.keys())
# Create list of focal ants
focal = [106, 63, 23, 53, 19]
# Create list of lists of all other antIDs excluding each focal ID
# others = [list(set(ants) - set([x])) for x in focal]

In [ ]:
# Experimental phase
day_starts_exp = [
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]
exp_days = [1, 2, 3, 4, 5]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control Phase
day_starts_control = list(
    np.repeat(datetime(2022, 5, 1, 15, 54).astimezone(tz=None), 5)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre experimental phase
day_starts_pre = [
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase
day_starts_post = [
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel42_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel42"
)

In [ ]:
cfel42_enc.to_csv("Cfel42_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 1

In [ ]:
f_myrmidon = (
    "/media/egeorge/Elements/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
)
exp = fm.Experiment.Open(f_myrmidon)
# Thresholds
encounter_threshold = 300  # threshold for counting as an encounter
away_threshold = 1000  # Threshold for counting as the end of an encounter
# Focal Ants
focal = [87, 37, 58, 38, 3]
exp_days = [1, 2, 3, 4, 5]
# Experimental Phase list
day_starts_exp = [
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2022, 6, 4, 14, 48).astimezone(tz=None), 5)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel1_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel1"
)

In [ ]:
cfel1_enc.to_csv("Cfel1_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 54

In [ ]:
f_myrmidon = (
    "/media/egeorge/Elements/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
)
exp = fm.Experiment.Open(f_myrmidon)
# Thresholds
encounter_threshold = 300  # threshold for counting as an encounter
away_threshold = 1000  # Threshold for counting as the end of an encounter
# Focal Ants
focal = [108, 114, 62, 12]
exp_days = [1, 2, 3, 4]
# Experimental Phase list
day_starts_exp = [
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2022, 6, 19, 14, 25).astimezone(tz=None), 4)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel54_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel54"
)

In [ ]:
cfel54_enc.to_csv("Cfel54_AllAnts_Focal_Encounters.csv", index=False)